<img style="float: right;width: 100px" src="https://www.enib.fr/images/logo-enib-accueil.jpg">

<div>
    <p><h3>ASN (S6)</h3></p>
    <p>Année 2020</p>
    <p><em>{{Nom}} {{Prénom}}</em></p>
</div>

<div style="text-align: center;padding-bottom:20px;padding-top:10px"><h1>Asservissements Numériques</h1>
    <h2>Comportements des Systèmes échantillonnés du 1ier et 2ieme ordre</h2>
</div>    

In [ ]:
import numpy as np
from ipywidgets import interact
from ENIB_control import figure
from control import tf, c2d, feedback, bode

## 1. But de l'étude

Le comportement des systèmes continus (analogiques) simples (1er et 2ème ordre) est bien connu : que devient-il lorsque la commande est du type échantillonné ? Cette étude concerne l'analyse de l'asservissement numérique de processus continus du 1er et du 2nd ordre. Il s'agit d'un travail de simulation en utilisant la librairie `python-control`

Ce TP comporte deux parties :
* une introduction aux fonctions de transfert numériques sous la forme d'une préparation théorique à effectuer avant le TP.
* une validation des résultats théoriques par simulation sur un Jupyter Notebook (3 h).

## 2. Préparation

### 2.1 Rappel de cours

Un asservissement numérique est généralement un système hybride, composé d'un processus analogique dont on souhaite contrôler la sortie $s(t)$ et d'un système de commande de type numérique (PC, carte microcontrôleur,...) dont le rôle est de synthétiser la commande du processus analogique. La figure 1 suivante présente le schéma typique d'un asservissement numérique.

<figure style="padding:30px">
  <img src="./img/labo3_1.png" style="width: 600px" />
  <figcaption style="text-align:center">Fig.1 - Asservissement numérique.</figcaption>
</figure>

Des éléments de conversion analogique-numérique (CAN) et numérique-analogique (CNA) sont nécessaires pour passer d'un domaine à un autre. Les systèmes hybrides mélangeant du numérique et de l'analogique sont généralement traités par les fonctions de transfert discrètes (Transformée en $Z$). Il est donc nécessaire d'une part, de connaître la fonction de transfert discrète équivalente à une fonction de transfert continue et d'autre part, de prendre en compte l'effet des CNA ou CAN.
* Le convertisseur CAN réel est modélisé par un échantillonneur idéal suivi d'un gain de conversion.
* Le convertisseur CNA réel est modélisé par un bloqueur d'ordre zéro (BOZ) suivi d'un gain de conversion.

<figure style="padding:30px">
  <img src="./img/labo3_2.png" style="width: 800px" />
  <figcaption style="text-align:center">Fig.2 - Modélisation d'un CAN réel et d'un CNA réel.</figcaption>
</figure>

Dans ces schémas le signal numérique N(t) est assimilé à un signal échantillonné. Les signaux numériques sont traités comme des signaux échantillonnés. On néglige l'étape de quantification. Dans le cours, il est démontré que le passage d'une fonction de transfert continue à une fonction de transfert discrète en tenant compte du CAN muni d'un BOZ est donné par la relation suivante :

<figure style="padding:30px">
  <img src="./img/labo3_3.svg" style="width: 500px" />
</figure>

$$F(z)=\frac{z-1}{z}TZ\left[TL^{-1}\left(\frac{F(p)}{p}\right)\right]$$

### 2.2 Étude d'un système du 1er ordre

On considère un système du premier ordre continu incorporé dans une boucle avec échantillonnage et bloqueur d'ordre 0 (BOZ). 

<figure style="padding:30px">
  <img src="./img/labo3_4.svg" style="width: 500px" />
   <figcaption style="text-align:center">Fig.3 - Système en boucle fermée.</figcaption>
</figure>

La période d'échantillonnage est fixée à $T_e=10$ ms et la fonction de transfert du système continu est égal à 

$$F(p)=\frac{K}{1+0.1p}$$

#### Question a)

Calculez la fonction de transfert en boucle ouverte $F(z) = S(z)/\epsilon(z)$ en utilisant la relation vue dans la partie 2.1.

$$F(z)= \frac{0.0952K}{z − 0.9048}$$

#### Question b)

Donnez la plage de valeurs admissibles pour $K$ assurant la stabilité de l'asservissement. **On rappelle qu’un système échantillonné est stable si le module de tous ses pôles est inférieur à l’unité.**

$$−1 < K < 20$$

#### Question c) 

En boucle fermée et pour $K=1$ :
* Écrivez l'équation de récurrence liant les échantillons successifs de $s[n]$ et de $e[n]$; déduisez en $s[n]$, valeur de la sortie à l'instant $t = n T_e$. En supposant $s[0]=0$ et une entrée de type échelon unitaire ($e[n] = 1$ si $n\ge  0$).
* Calculez quelques valeurs numériques.
* Calculez également la valeur finale de $s[n]$ pour $n\to \infty$.

$$s[n] = 0.0952e[n − 1] + 0.8096s[n − 1]$$

* s[0]=0, s[1]= 0.0952, s[2] = 0.1723, s[3]= 0.2347.
* $s[\infty]=H(1)=0.5$. 

#### Question d):  Analyse fréquentielle de la F.T. échantillonnée

La réponse fréquentielle est obtenue en étudiant le gain et la phase de la fonction complexe $F(j\omega)$ correspondant à $F(z)$ où on remplace $z$ par $e^{j\omega T_e}$ ($\omega$ pulsation réelle du signal d'entrée). Pour respecter le théorème de Shannon, on s'intéresse uniquement à la courbe de réponse résultant de la variation de $0$ à $\omega_e/2$ (la fréquence varie de $0$ à $F_e/2$).
* Donnez les expressions du module et de la phase de $F(z)$ en posant $z =e^{j\omega T_e}$.

La réponse fréquentielle s'obtient en posant $z=e^{j\omega T_e}$ où $\omega$ correspond à la pulsation réelle en rad/s. Nous obtenons alors 

\begin{align}
F(e^{j\omega T_e})&=\frac{a_0}{e^{j\omega T_e}+b_0}=\frac{a_0}{\cos(\omega T_e)+j \sin(\omega T_e)+b_0}
\end{align}
avec $a_0=0.0952K$ et $b_0=-0.9048$.

\begin{itemize}
\item Expression du module. En exploitant les propriétés du module, nous obtenons
\begin{align}
|F(e^{j\omega T_e})|&=\frac{|a_0|}{\sqrt{(\cos(\omega T_e)+b_0)^2+\sin^2(\omega T_e)}}
\end{align}
\item Expression de l'argument. En exploitant les propriétés de l'argument, nous obtenons
\begin{align}
\arg[F(e^{j\omega T_e})]&=\arg[a_0]- \arctan\left[\frac{\sin(\omega T_e)}{\cos(\omega T_e)+b_0}\right]
\end{align}

### 2.3 Étude d'un système du 2nd ordre

Soit un système continu du 2ème ordre tel que :

$$F(p)\frac{5}{1+0.08p+0.01p^2}$$

Ce système est commandé par un dispositif échantillonneur bloqueur (BOZ) à la cadence $T_e = 0.15$s.

#### Question a) 

Exprimez les pôles de $F(p)$ sous la forme $a+jb$.

Les pôles du système s'obtiennent à partir de l'équation

\begin{align}
0.01p^2+0.08p+1=0
\end{align}
Le discriminant est égal à $\Delta = 0.08^2-4\times 0.01= -0.0336$. Nous obtenons donc deux pôles complexes-conjugués:

\begin{align}
p_1&= \frac{1}{2\times 0.01}\times (-0.08+j\sqrt{0.0336})=-4+9.1652j\\
p_2&=\frac{1}{2\times 0.01}\times (-0.08+j\sqrt{0.0336}) =-4-9.1652j
\end{align}

#### Question b) 

Déduisez-en les pôles de $F(z)$ sachant que $z = e^{T_e p}$ (faire les calculs numériques).

Les pôles de la FT échantillonnés s'obtiennent via l'expression $p_z = e^{pT_e}$. Nous obtenons donc

\begin{align}
p_{z1}&= 0.1069 + 0.5383i\\
p_{z2}&=0.1069 - 0.5383i
\end{align}

#### Question c) 

Déterminez le temps de réponse prévisible de la sortie échantillonnée lorsque l'entrée est un échelon unité.

Le $t_r$ d'un système échantillonné à la période $T_e$ est le même que celui du signal continu équivalent (à $T_e$ près).  Le facteur d'amortissement et la pulsation naturelle sont identiques à ceux du système continu.

Nous obtenons alors 
\begin{align}
\omega_n & =10~rad/s\\
m&=0.4
\end{align}
et un temps de réponse environ égal à $t_r=0.77$s (à $T_e/2$ près).

## 3. Validation des résultats établis sur le 1er ordre

La validation des résultats établis en préparation se fera via le package `python-control`. L'ensemble des fonctions nécessaires et leur syntaxe sont décrits dans la documentation du package. La fonction de transfert analysée est celle de la partie §2.2.



### 3.1 FT échantillonnée $F(z)$ avec BOZ

> Le package `python control`permet de calculer directement la FT discrète à partir de la FT continue avec la commande `c2d` avec ou sans bloqueur suivant les options choisies (voir [doc](https://python-control.readthedocs.io/en/0.8.3/generated/control.matlab.c2d.html) ). La fonction de transfert discrète du dispositif avec bloqueur à $T_e = 10$ ms se calcule par : `fz = c2d(f, 0.01, 'zoh')` ou `fz = c2d(f, 0.01)`.



* Saisissez la fonction de transfert continue $F(p)$ : `f = tf(K, [0.1,1])`  avec $K=1$.
* Calculez la FT échantillonnée $F(z)$ correspondant au système avec BOZ en utilisant la commande `c2d`. 
* Validez les résultats trouvés dans la préparation.
* Vérifiez les informations concernant les FTBO $F(z)$ et $F(p)$.

In [ ]:
f = tf(1, [0.1,1])
fz = c2d(f,0.01)
print(fz)

### 3.2 Réponses indicielles

* Tracez sur le même graphique les réponses indicielles de $F(p)$ et $F(z)$.
* Déterminez les caractéristiques principales des réponses indicielles. Utilisez la fonction `step_info` pour la réponse à un échelon et la fonction `damp` pour le calcul de l'amortissement. Expliquez la forme en "escalier" de la réponse de $F(z)$.
* Comparez aux performances prévisibles déterminées à partir des propriétés connues des FT.

In [ ]:
fig = figure("time")
fig.plot(f)
fig.plot(fz)
fig.show()

### 3.3 Fonction de transfert en boucle fermée

* Déterminez la FTBF $H(z)$ (fonction `feedback`) pour un retour unitaire et pour $K=1$. 
* Simulez la réponse en boucle fermée à un échelon. Vérifiez les valeurs établies dans la partie §2.2.

In [ ]:
hz = feedback(fz,1)
print(hz)

### 3.4  Analyse fréquentielle de la F.T. échantillonnée

La réponse fréquentielle est obtenue en étudiant le gain et la phase de la fonction complexe $F(j\omega)$ correspondant à $F(z)$ où on remplace $z$ par $e^{j\omega Te}$ ($\omega$ pulsation réelle du signal d'entrée). On s'intéresse uniquement à la courbe de réponse résultant de la variation de $0$ à $\omega_e/2$ (la fréquence varie de $0$ à $Fe/2$). La réponse fréquentielle sera visualisée à l'aide du diagramme de Black. 

* Tracez le lieu de Black avec la commande `nichols`.
* Comparez aux résultats obtenus dans la préparation pour $\omega=0$ et $\omega =\omega_e/2$. 
* Retrouvez la condition de stabilité (méthode graphique).
* Déterminez les marges de gain et de phase à partir du plan de Black Nichols.

In [ ]:
fig = figure("nichols")
fig.plot(fz)
fig.grid(show_phase=False)
fig.show()

Créez des nouvelles FTBO `fz*5` et `fz*15`. Pour les gains`$K=5$ et $K=15$ : 

* tracez le lieu de Black des différentes FT sur une même courbe
* déterminez les marges de gain et de phase,
* déterminez le facteur de résonance de la FTBF.

In [ ]:
fig = figure("nichols")
fig.plot(fz)
fig.plot(5*fz)
fig.plot(15*fz)
fig.grid(show_phase=False)
fig.show()

### 3.5 Transitoire associé à un pôle réel

<div class='alert alert-info'>
Les abaques de second ordre sont disponibles à l'adresse : <a href="https://vincentchoqueuse.github.io/web_app_2nd_order_performances/index.html">https://vincentchoqueuse.github.io/web_app_2nd_order_performances/index.html</a>
</div>

Le mode transitoire d’un système continu est oscillant s’il possède des pôles complexes. Par contre pour un système échantillonné, même un pôle réel peut donner lieu à un mode transitoire oscillant.

Pour les gains $K= 5$ et $K = 15$ :
* calculez, le pôle de la FTBF
* simulez la réponse du système bouclé à un échelon.
* en reprenant les résultats de l’analyse fréquentielle (3.4), retrouvez les valeurs de dépassements $D\%$.

In [ ]:
sys_list_BF = []

fig = figure("time")
for k in [1,5,15]:
    hz = feedback(k*fz,1)
    
    print("K={} - poles: {}".format(k,hz.pole()))
    fig.plot(hz,label="K={}".format(k))
    
fig.show()   

La fonction `damp` donne l’amortissement équivalent des pôles d’une fonction de transfert.`

In [ ]:
def damp(sys):

    for pole in sys.pole():
        pole = pole.astype(complex) # WTF: the python control "damp" function is buggy due to this missing cast !
        pole_continuous = np.log(pole)/sys.dt
        wn = np.abs(pole_continuous)
        m = -np.real(pole_continuous)/wn
        print("dpoles {:.3f} : wn={:.3f} rad/s, m= {:.3f}".format(pole, wn, m))


* Déterminez l’amortissement des 3 FTBF.
* Tracez dans le plan complexe les pôles de $H(z) = K.F(z)/ (1+K.F(z))$ à l'aide de la commande commandes `rlocus(fz)`. Les pôles de $H(z)$ sont représentés pour $K$ variant de $0$ à l'infini. Retrouvez les pôles de $H(z)$ pour les gains $K = 1$, $K= 5$ et $K = 15$
* Retrouvez les valeurs admissibles de $K$ assurant la stabilité de l'asservissement trouvés dans la préparation. (Proposez une méthode pour les valeurs négatives de $K$.)
* Expliquez le mode oscillant.

In [ ]:
for k in [1,5,15]:
    hz = feedback(k*fz,1)
    damp(hz)
    
fig = figure("rlocus")
fig.plot(fz,k_vect=np.logspace(-2,1.2,1000))
fig.show()

### 3.6 Synthèse d'un correcteur proportionnel par placement des pôles

L'objectif est ici de déterminer le gain $K$ à ajouter dans la boucle ouverte pour obtenir un amortissement en boucle fermée de $m = 0.45$ en travaillant directement dans le plan complexe (`rlocus`).

* Déterminer la valeur de $K$ via la fonction `rlocus`.
* Vérifiez que vous obtenez le même résultat en travaillant dans le plan de Black-Nichols.
* Calculez avec Python, $H(z)$, la fonction de transfert du système bouclée et corrigée avec le gain $K$ ainsi trouvé : `h=feedback(K*f,1)`
* Tracez la réponse indicielle de $H(z)$. Déterminez les caractéristiques de cette réponse ($tr$, $D\%$, $m$, $s[\infty]$).

In [ ]:
K = 11.65

h=feedback(K*fz,1)

fig = figure("time")
fig.plot(h)
fig.show()

## 4. Étude d'un système du 2nd ordre

Soit le système continu du 2ème ordre tel que :
$$F(p)=\frac{5}{1+0.08p+0,01p^2}$$

Ce système est commandé par un dispositif échantillonneur bloqueur (BOZ) à la cadence $T_e = 0.15$s.


### 4.1 Analyse de la fonction de transfert.

* Déterminez les pôles de $F(p)$ en utilisant l'instruction `pole`.
* Calculez la FTBO $F(z)$ de l'ensemble système + BOZ. (`c2d`) Notez la présence d'un "zéro".

In [ ]:
F = tf([5],[0.01,0.08,1])
print(F.pole())

Te = 0.15
Fz = c2d(F,Te)
print(Fz)

### 4.2 Stabilité

Ce système, bouclé par un retour unitaire, sera-t-il stable ?

In [ ]:
fig = figure("nichols")
fig.plot(Fz)
fig.grid(show_phase=False)
fig.show()

On laisse le point critique sur notre gauche $->$ système instable en BF.

### 4.3 Réponse indicielle en BO

Connaissant les paramètres du système continu (amortissement $m$ et pulsation libre $\omega_n$) on peut prévoir l'allure de la réponse transitoire dans le cas d'une commande continue.

* Chiffrez, à partir des abaques, le premier dépassement relatif ($D\%$) et le temps de réponse (à $\pm 5\%$ de la valeur finale).
* Tracez la réponse indicielle échantillonnée et mesurez ses caractéristiques ; comparez les valeurs.

### 4.4 Influence du "zéro"

Un système continu, d'amortissement $m$ et de pulsation libre $\omega_n$, commandé par un échantillonneur bloqueur, est décrit par une FT en $z$ caractérisée, à $T_e$ donnée, par:
* des pôles liés à ceux de la FT continue
* un "zéro", fonction de $m$, $\omega_n$, $T_e$ appelé "zéro" optimal.

On se pose la question suivante : à pôles donnés, quelle est l'influence du "zéro" sur le régime transitoire ? Ceci est important dans la mesure où, pour la synthèse de correcteurs, on se contente souvent de "placer" uniquement les pôles.

* Saisissez deux nouvelles FT en $z$, de mêmes pôles que $F(z)$, mais dont on fixe le "zéro" à $+50\%$, puis $-50\%$ du "zéro" de $F(z)$. 

> Attention : veillez à conserver le même gain statique, c'est à dire même valeur pour $F(z)$ avec $z=1$.

* Simulez la réponse à un échelon (BO). Conclusion.
* Simulez la réponse à un échelon en changeant simplement le signe du "zéro" optimal.
* Donnez pour l'ensemble des FT l'amortissement équivalent d'un second ordre continu (aidez vous des abaques) et le temps de réponse à $\pm 5% de la valeur finale.